# Axis of interest

In [1]:
import os
import json
import lmstudio as lms
import sys
import re
from langchain_openai import ChatOpenAI
from pathlib import Path

repo_root = Path(__file__).resolve().parents[1] if "__file__" in globals() else Path("/Users/jpconde/Facultad/tesis")
if str(repo_root) not in sys.path:
    sys.path.append(str(repo_root))

In [2]:
from axis_of_interest.schemas import list_of_aoi, indice_aoi
from axis_of_interest.prompts import template_prompt_generate_plot_schema, template_prompt_generate_cuento, tenplate_prompt_generar_axis_of_interest
from axis_of_interest.utils import render_aoi_md, render_plot_schema_md

### Crear AoI en python

Pasamos los AoI del Dr.Gervás a python. Usamos pydantic


In [ ]:
for aoi in list_of_aoi:
    print(render_aoi_md(aoi))
    print("\n")

AXISofINTEREST =  CALL TO ACTION REWARD
PROTAGONIST =  called
ROLES =  called caller rewarded
────────────────────────────────────────────────────────────────
PLOT-SPAN-NAME =  Call

  CallToAction  characters(called=Hero,caller=Sender)

────────────────────────────────────────────────────────────────
PLOT-SPAN-NAME =  Reward

  Reward  characters(rewarded=Hero)

────────────────────────────────────────────────────────────────



AXISofINTEREST =  CONFLICT
PROTAGONIST =  attacker
ROLES =  attacker defender winner loser
────────────────────────────────────────────────────────────────
PLOT-SPAN-NAME =  Struggle

  Struggle  characters(attacker=X,defender=Y)

────────────────────────────────────────────────────────────────
PLOT-SPAN-NAME =  Victory

  Victory  characters(winner=X,loser=Y)

────────────────────────────────────────────────────────────────



AXISofINTEREST =  DONOR
PROTAGONIST =  tested
ROLES =  tested tester user gift
───────────────────────────────────────────────────────

### Crear Plot Schema

Le pegamos a una api de un LLM, con algunos axes of interest en el prompt y le pedimos que nos genere un Plot Schema

In [4]:
print(indice_aoi)

{'CALL TO ACTION REWARD': 0, 'CONFLICT': 1, 'DONOR': 2, 'INTERDICTION VIOLATED': 3, 'JOURNEY': 4, 'KIDNAPPING': 5, 'KIDNAPPING PUNISHMENT': 6, 'PURSUIT': 7, 'RAGS 2 RICHES': 8, 'RELENTING GUARDIAN': 9, 'REPENTANCE': 10, 'RIVALRY': 11, 'SHIFTING LOVE': 12, 'TASK': 13, 'UNRELENTING GUARDIAN': 14}


In [5]:
model1 = ChatOpenAI(
    model="gpt-5",
    temperature=0.0,
)

In [ ]:
lista1 = [list_of_aoi[indice_aoi["RAGS 2 RICHES"]], list_of_aoi[indice_aoi["CONFLICT"]], list_of_aoi[indice_aoi["DONOR"]]]

prompt_generate_plot_schema = template_prompt_generate_plot_schema.replace(
    "{axis_of_interest}",
    json.dumps([a.model_dump() for a in lista1], indent=2)
)

result1 = model1.invoke(prompt_generate_plot_schema)

In [6]:
print(render_plot_schema_md(result1.content))

PLOT-SCHEMA =  RAGS2RICHES-DONOR-CONFLICT
DESCRIPTION:
Trama compuesta: un héroe en la miseria aspira a más, es probado por un donante, recibe y usa un amuleto, se transforma, enfrenta a un villano y alcanza la recompensa.
────────────────────────────────────────────────────────────────
AXISofINTEREST =  RAGS 2 RICHES
PLOT-SPAN-NAME =  Rags
DESCRIPTION:
El héroe sufre la pobreza y la carencia inicial.

Poverty  characters(sufferer=Hero)
  — Hero padece necesidad material y simbólica; punto de partida humilde.

────────────────────────────────────────────────────────────────
AXISofINTEREST =  RAGS 2 RICHES
PLOT-SPAN-NAME =  Aspiration
DESCRIPTION:
El héroe formula un deseo explícito de superación.

Aspiration  characters(aspirer=Hero)
  — Hero fija una meta: salir de la miseria y ganar dignidad.

────────────────────────────────────────────────────────────────
AXISofINTEREST =  DONOR
PLOT-SPAN-NAME =  Tested
DESCRIPTION:
El donante pone a prueba al héroe; si la supera, le concede un age

### Crear Cuento Completo

Dado un Plot Schema usamos un LLM para generar un cuento

In [7]:
prompt_generate_cuento = template_prompt_generate_cuento.replace(
    "{plot_schema}",
    result1.content
)

In [8]:
cuento = model1.invoke(prompt_generate_cuento)
print(cuento.content)

En los barrios bajos de San Velorio, donde las aceras eran puertos de sombras y los techos rezumaban la sal de un mar invisible, vivía Mateo, encuadernador ambulante. Cosía lomos con hilo rescatado y pegamento aguado, y las portadas, de cartón desparejo, parecían querer cerrarse para siempre sobre historias a las que faltaban páginas. Si el hambre tenía música, en la casa de Mateo sonaba a cucharas golpeando fondo vacío y a viento silbando por rendijas. Su madre, ya muerta, le había dejado una cajita con agujas y un consejo: “Ten manos firmes y corazón claro.” Lo primero lo aprendió a fuerza de frío; lo segundo, a fuerza de necesidad.

Cada mañana pasaba frente al Palacio de los Arcedianos, donde las ventanas reflejaban un sol dorado que en su barrio parecía de cobre gastado. Miraba aquellas losas brillantes y pensaba que el mundo era un libro con tapas que él no podía abrir. No envidiaba el oro; codiciaba, apenas, la dignidad de caminar por la plaza sin que lo rozara la sospecha del g

### Crear AoI dado un cuento

Acá lo que hacemos es el paso inverso. Queremos mandar un cuento y ver si el LLM es capaz de detectar los AoI. Hicimos dos experimentos:
- Con el cuento que creamos recien y ver si nos devuelve los AoI que usamos
- Con un cuento cualquiera, este cuento nosotros lo analizamos nosotros y detectamos algunos AoI

Lo deberia poder hacer bien el LLM

In [6]:
cuento1='''"Eddie, a self-reflective sophomore at Groton High, navigates his way through the monotonous routine of long chemistry lectures and intricate English literature workshops.",
      "His best companion is not a fellow classmate, but a weathered 1950s Gibson acoustic guitar.",
      "Eddie's fingers caress the frets, breathing life into the worn-out instrument, putting forth his raw emotions in the form of harmonious resonance.",
      "After another day of blending in with the background at school, Eddie retreats to his secret sanctuary - an antiquated, barely-used storage room heaving with dusty crates and paint cans of faded colors.",
      "With just a week remaining before the Groton High Talent Show, Eddie engrosses himself into perfecting a unique acoustic rendition of LMFAO's upbeat dance track, \"Sexy and I Know It\".",
      "As he finally takes the center stage in the school auditorium, he strikes a chord that encapsulates the audience within his passion-filled bubble of music.",
      "Caught amidst the crowd brimming with glaring letterman jackets and twinkling cheerleading skirts, Sarah, the popular cheerleader, watches Eddie.",
      "Her effusive clapping for Eddie's one-of-a-kind performance compels others to question their preconceived notions towards him.",
      "Intrigued by Eddie's raw talent and unconventional choice in music, Sarah approaches Eddie by his locker in an effort to bridge their worlds.",
      "Overwhelmed by Sarah's sudden openness, Eddie finally accepts her offer ' a partnership in creating a musical act for the highly anticipated Groton Town Day festival.",
      "As the days pass by, Eddie and Sarah's late afternoon jamming sessions in the old storage room form an integral part of their daily routine.",
      "Their shared love for music weaves a bond beyond the titular differences in their social stature.",
      "Sarah introduces him to the world of pep rallies, cheer-leading practice, school popularity, and acceptance.",
      "Their unexpected alliance noticeably stirs the otherwise peaceful town of Groton.",
      "Eddie's sudden surge in popularity strikes a discordant note with Andy, the star quarterback of Groton High, thus resulting in a growing rift in their erstwhile strong friendship.",
      "Their shattering friendship adds fuel to the rumors, increasingly creating misunderstandings that are based on hearsay.",
      "The football coach carefully lays a trail of lies to pit him against Eddie.",
      "He feeds Andy with distorted tales of Eddie threatening their football glory by garnering attention away from their Friday-night-football games.",
      "Succumbing to his coach's manipulation and the loss of his most trusted friend, Eddie finds himself stranded on an island of estrangement.",
      "Sarah channels her effort into rekindling his enthusiasm for music to guide him out of his ongoing despair.",
      "Eddie, filled with a new-found determination and conviction, confronts Andy in the football locker room.",
      "In the heat of the moment, Andy realizes his mistake and guilt clouds his face.",
      "He offers an earnest apology to Eddie, bringing back a glimmer of their old camaraderie.",
      "With newfound unity, they expose the coach's unethical manipulation in front of the school committee.",
      "Meanwhile, Sarah voices her love for Eddie, fueling him with a special kind of courage that only love can bring.",
      "Eddie reciprocates her feelings and makes a quiet promise to give their flourishing feelings a chance.",
      "With love and upbeat melodies now coloring his world in bright hues, Eddie nervously anticipates the approaching festival, a platform to showcase his skills and validate his passion.",
      "The festival day wields an air of excitement, and the small town of Groton buzzes with an unusual fervor as Eddie and Sarah captivate the audience with their performances.",
      "Eddie's public proclamation of love for Sarah sends waves of cheer, whistles, and applause throughout the crowd.",
      "The crowd, moved by Eddie's musical prowess and charm, dispels the football coach's influence and expresses their disdain for his manipulative attitude.",
      "The coach is forced to relinquish his position.",
      "The once divided town of Groton unites to commemorate the invigorating power of music, the boldness to embrace love, and the significance of loyalty.",
      "The change is palpable as even Andy, a character who was initially a symbol of rivalry, is glimpsed amongst the crowd, clapping his hands and sharing in the excitement.",
      "As Eddie plucks the last string on his Gibson, the screen seamlessly merges into black."'''

Cuento de terror que yo no le veo una separación clara de AoI 

In [7]:
cuento2 = """ "An old man conducts a ritual by slaughtering a chicken and hurriedly spreading ash in a circle.",
      "His sickly wife appears at the door and tells him to stop, to which he replies \"Burn in hell\".",
      "He starts reading from a book and the letters start to bleed.",
      "Then a little girl whispers his name, Yosille, and asks him if he wants to see his wife again, to which he replies negatively.",
      "He steps outside the circle, swallowing a dark smoke as his sclera turns black.",
      "He then stabs himself with a knife from his back pocket.",
      "The demon tries to remove the amulet around his neck to no avail.",
      "Arthur (Art) and his pregnant wife come to Brooklyn to visit his father, who runs a Jewish funeral house in a Hasidic community.",
      "He and his father, Saul, became estranged after he married Claire, a non-Jewish British food journalist.",
      "Saul is welcoming to Art and Claire, wanting to establish a relationship with both of them.",
      "Art has an alternative agenda for the visit.",
      "He needs his father to mortgage his house to help his failing real estate business.",
      "Art volunteers to help his father with a new suicide body that has come in.",
      "The funeral home is the basement of their home.",
      "Art has a run in with Heimish, who works with his father.",
      "Heimish cares for Saul and is suspicious of Art's visit.",
      "As Saul and Heimish speak, Arthur prepares the body of Yosille.",
      "While preparing the body, the amulet around his neck falls through a drain, releasing the demon which immediately begins to haunt Claire.",
      "Saul comes across the knife that Yosille used to commit suicide and becomes suspicious.",
      "He calls Reb Chayim and asks about the inscription.",
      "Reb Chayim tells Saul that this, in conjunction with an amulet is used for a binding spell.",
      "But as there was no amulet found, Chayim dismisses its importance.",
      "Meanwhile, Art and Heimish go to pick up another body.",
      "It is the body of Sarah Scheindel, a missing girl from the community.",
      "Heimish picks up Art's phone and discovers Art's money problems and his intention to ask his father to mortgage his house.",
      "Heimish exposes Art in front of Saul and Claire.",
      "When Claire confronts Art, he opens up to her about his loss of faith after the death of his mother.",
      "That night, while Saul is in the mortuary, the demon attacks and kills him.",
      "Art wakes up the next day to see men praying around his father's body.",
      "Heimish blames Art saying he died of a broken heart and gives him the paper that Art needed Saul to sign.",
      "During the funeral both Art and Claire have disturbing visions.",
      "Claire has visions of Sarah Schneidel.",
      "Art sleepwalks and draws a sigil.",
      "He wakes up on a carpet and thinks he was dreaming.",
      "Art struggles to wrap up his father's business as Heimish has quit.",
      "He is told that Yosille needs a police clearance before he can be buried or a next-of-kin must sign off on it, even though it was a clear suicide.",
      "He goes to Yosille's house where he finds a video tape.",
      "He discovers Yosille was attempting to conjure the archangel of life, Martiel, so that he could bring back his dead wife Aida.",
      "He used Sarah for the ritual.",
      "But the ritual killed Sarah and conjured the demon Abyzou instead.",
      "Meanwhile, \u201cAida\u201d comes to visit Claire and tells her she wants to see her husband's body.",
      "When Claire tells her husband Aida is here to see Yosille's body, Art realizes his wife is in danger.",
      "While Claire is on the phone, \u201cAida\u201d burns Yosille's body, which releases the Abyzou.",
      "Art tries to rescue his wife, but is unable to.",
      "Art turns to Heimish and shows him Yosille's video.",
      "Heimish brings in Chayim for help.",
      "They discover the sigil and Chayim says that the demon gains strength and is fed when a child steps in it.",
      "Chayim explains the binding ritual and explains that Abyzou has to be trapped in a body.",
      "Art is willing to sacrifice himself for his wife and child.",
      "Abyzou kills Chayim, so Art and Heimish rush to begin the ritual.",
      "Heimish tells Art not to step outside the circle, but Heimish is taken by the demon.",
      "Art begins the ritual, but he sees his wife who is taken and threatened by the demon.",
      "He leaves the circle to rescue his wife who is being held against a high window.",
      "Unable to do anything else, Art finishes the inscription to force the demon inside him, but he is unable to kill himself.",
      "Heimish crawls to him and Art begs Heimish to stab him with the inscripted knife.",
      "Heimish reluctantly stabs Art, then tells him that he should have stayed in the circle.",
      "Heimish alludes to the amulet that has been taken off and flung across the room.",
      "Art then sees the real Heimish's dead body in the next room, and dies helplessly.",
      "Claire wakes up under a table.",
      "She rushes to the demon masquerading as Art, but looks down to see she is standing in the sigil.",
      "Abyzou appears and takes her." """

In [9]:
from langchain_core.prompts import PromptTemplate

template_prompt_devolver_aoi = PromptTemplate.from_template("Dado el siguiente cuento: {cuento}, y la siguiente lista de Axis of Interest: {axis_of_interest}, devuelve una lista con los nombres de los Axis of Interest que abarquen la historia. Tambien el orden en que se entrelazan los distintos Plot Spans.")
prompt_devolver_aoi = template_prompt_devolver_aoi.invoke({"cuento": cuento1, "axis_of_interest": json.dumps([a.model_dump() for a in list_of_aoi], indent=2)})


In [10]:
aoi_devuelto = model1.invoke(prompt_devolver_aoi)

In [11]:
print(aoi_devuelto.content)

- Axis of Interest presentes en la historia:
  - CALL TO ACTION REWARD
  - CONFLICT
  - DONOR
  - RAGS 2 RICHES
  - RIVALRY
  - REPENTANCE
  - TASK

- Orden intercalado de los Plot Spans a lo largo de la narración:
  1) RAGS 2 RICHES — Rags
  2) RAGS 2 RICHES — Aspiration
  3) TASK — TaskSet (talent show)
  4) TASK — TaskSolved (talent show)
  5) CALL TO ACTION REWARD — Call (Sarah propone actuar juntos en el festival)
  6) TASK — TaskSet (preparar el número para Groton Town Day)
  7) RAGS 2 RICHES — Transformation (sube su estatus al vincularse con Sarah)
  8) RIVALRY — RivalryDeclared (se rompe la amistad con Andy)
  9) CONFLICT — Struggle (el coach manipula a Andy contra Eddie)
  10) DONOR — Tested (Sarah lo pone a prueba/lo reanima; reacción de Eddie)
  11) DONOR — UseOfAMagicalAgent (Eddie usa el “don”: confianza/pareja musical)
  12) REPENTANCE — Transformation (Andy toma conciencia del error)
  13) REPENTANCE — Repentance (Andy se disculpa)
  14) RIVALRY — Cooperation (Eddie y A

In [ ]:
prompt_devolver_aoi2 = template_prompt_devolver_aoi.invoke({"cuento": cuento1, "axis_of_interest": json.dumps([a.model_dump() for a in list_of_aoi], indent=2)})
aoi_devuelto2 = model1.invoke(prompt_devolver_aoi2)

In [17]:
print(aoi_devuelto2.content)

Ejes de interés presentes en la historia:
- RAGS 2 RICHES
- SHIFTING LOVE
- CALL TO ACTION REWARD
- TASK
- RIVALRY
- CONFLICT
- REPENTANCE

Orden en que se entrelazan los Plot Spans:
1) RAGS 2 RICHES — Rags
2) SHIFTING LOVE — FirstMeeting
3) CALL TO ACTION REWARD — Call
4) TASK — TaskSet
5) RAGS 2 RICHES — Aspiration
6) RAGS 2 RICHES — Transformation
7) RIVALRY — RivalryDeclared
8) SHIFTING LOVE — LoveShift
9) CONFLICT — Struggle
10) REPENTANCE — Transformation
11) REPENTANCE — Repentance
12) RIVALRY — Reconciliation
13) RIVALRY — Cooperation
14) TASK — TaskSolved
15) SHIFTING LOVE — Reconciliation
16) CONFLICT — Victory
17) CALL TO ACTION REWARD — Reward
18) RAGS 2 RICHES — Riches


In [18]:
prompt_devolver_aoi3 = template_prompt_devolver_aoi.invoke({"cuento": cuento2, "axis_of_interest": json.dumps([a.model_dump() for a in list_of_aoi], indent=2)})
aoi_devuelto3 = model1.invoke(prompt_devolver_aoi3)
print(aoi_devuelto3.content)

Axis of Interest presentes en la historia:
- CALL TO ACTION REWARD
- CONFLICT
- DONOR
- INTERDICTION VIOLATED
- JOURNEY
- KIDNAPPING
- PURSUIT
- RAGS 2 RICHES
- RIVALRY
- TASK

Orden en que se entrelazan los Plot Spans:
1) INTERDICTION VIOLATED — Interdiction (La esposa de Yosille le dice que pare)
2) INTERDICTION VIOLATED — InterdictionViolated (Yosille sale del círculo y se apuñala)
3) JOURNEY — Out (Art y Claire viajan a Brooklyn)
4) RAGS 2 RICHES — Rags (Se revela la precariedad económica de Art)
5) RAGS 2 RICHES — Aspiration (Art pretende que Saul hipoteque la casa)
6) RIVALRY — RivalryDeclared (Choque Art–Heimish; recelos y exposición)
7) PURSUIT — Chase (Cae el amuleto y Abyzou empieza a acosar, sobre todo a Claire)
8) CONFLICT — Struggle (Abyzou ataca; muerte de Saul)
9) RIVALRY — Cooperation (Art recurre a Heimish; traen a Reb Chayim)
10) TASK — TaskSet (Chayim define la tarea: ligar al demonio en un cuerpo y matarlo)
11) DONOR — Tested / Character’sReaction (El donante instru

In [19]:
prompt_devolver_aoi4 = template_prompt_devolver_aoi.invoke({"cuento": cuento2, "axis_of_interest": json.dumps([a.model_dump() for a in list_of_aoi], indent=2)})
aoi_devuelto4 = model1.invoke(prompt_devolver_aoi4)
print(aoi_devuelto4.content)

- Axis of Interest presentes en la historia:
  - CALL TO ACTION REWARD
  - CONFLICT
  - DONOR
  - INTERDICTION VIOLATED
  - JOURNEY
  - KIDNAPPING
  - PURSUIT
  - TASK

- Orden (cronológico) en que se entrelazan los Plot Spans:
  1) INTERDICTION VIOLATED – Interdiction (regla del círculo/amuletos en el rito de Yosille)
  2) INTERDICTION VIOLATED – InterdictionViolated (Yosille sale del círculo y se mata)
  3) JOURNEY – Out (Art y Claire viajan a Brooklyn)
  4) CALL TO ACTION REWARD – Call (Art se implica en la funeraria; luego, llamado mayor: proteger a Claire y al bebé)
  5) DONOR – Tested (Saul consulta a Reb Chayim; primera “prueba”/orientación)
  6) PURSUIT – Chase (Abyzou acosa a Claire y Art; muerte de Saul; visiones)
  7) TASK – TaskSet (Chayim fija la tarea: atar a Abyzou en un cuerpo mediante el ritual)
  8) INTERDICTION VIOLATED – Interdiction (Heimish advierte: no salgas del círculo)
  9) INTERDICTION VIOLATED – Absentation (Chayim muere; Heimish es tomado/ausente)
 10) DONO

### Crear pipline que haga todo esto de una 

Dar una lista de AoI que quiero en mi historia -> crear plot schema -> crear cuento -> ver si el LLM predice bien los AoI que usó para crear la historia

In [23]:
def pipeline(lista_aoi, modelo):
    prompt_generar_plot_schema = template_prompt_generate_plot_schema.replace(
        "{axis_of_interest}",
        json.dumps([a.model_dump() for a in lista_aoi], indent=2)
    )
    plot_schema = modelo.invoke(prompt_generar_plot_schema)

    prompt_generar_cuento = template_prompt_generate_cuento.replace(
        "{plot_schema}",
        plot_schema.content
    )
    cuento = modelo.invoke(prompt_generar_cuento)

    prompt_devolver_aoi = template_prompt_devolver_aoi.invoke({"cuento": cuento.content, "axis_of_interest": json.dumps([a.model_dump() for a in lista_aoi], indent=2)})

    aoi_devuelto = modelo.invoke(prompt_devolver_aoi)
    return plot_schema, cuento, aoi_devuelto

In [24]:
aoi_prueba2 = [list_of_aoi[indice_aoi["JOURNEY"]], list_of_aoi[indice_aoi["RIVALRY"]], list_of_aoi[indice_aoi["CALL TO ACTION REWARD"]], list_of_aoi[indice_aoi["REPENTANCE"]], list_of_aoi[indice_aoi["PURSUIT"]]]
model2 = ChatOpenAI(
    model="gpt-5-nano",
    temperature=0.0,
)

In [25]:
ps2, cuento2, aoi2 = pipeline(aoi_prueba2, model2)

In [28]:
print(render_plot_schema_md(ps2.content))


PLOT-SCHEMA =  PLOT SCHEMA NEURO-SIMBÓLICO
DESCRIPTION:
Plot schema que agrupa los ejes narrativos JOURNEY, RIVALRY, CALL TO ACTION REWARD, REPENTANCE y PURSUIT, con sus respectivos spans y plot atoms, siguiendo un enfoque neuro-simbólico para cohesión y trazados de relaciones de largo alcance.
────────────────────────────────────────────────────────────────
AXISofINTEREST =  JOURNEY
PLOT-SPAN-NAME =  Out
DESCRIPTION:
Etapa 'Out' dentro del eje 'Journey'.

Departure  characters(traveller=Hero)
  — Personajes: traveller=Hero. Ubicaciones: origin=Home, destination=Elsewhere.

────────────────────────────────────────────────────────────────
AXISofINTEREST =  JOURNEY
PLOT-SPAN-NAME =  Back
DESCRIPTION:
Etapa 'Back' dentro del eje 'Journey'.

Return  characters(traveller=Hero)
  — Personajes: traveller=Hero. Ubicaciones: origin=Elsewhere, destination=Home.

────────────────────────────────────────────────────────────────
AXISofINTEREST =  RIVALRY
PLOT-SPAN-NAME =  RivalryDeclared
DESCRIPTIO

In [29]:
print(cuento2.content)

Escena Out - Departure
El viajero conocido como Hero se despide de su hogar con una mezcla de temblor y curiosidad. La puerta golpea suavemente al cerrarse y, fuera, el mundo parece respirarlas en susurros: un mapa antiguo que late bajo la tela de la noche y una ruta que aún no entiende. Hero camina hacia Elsewhere, donde cada paso parece abrir un ojo nuevo en el paisaje. No lleva más que su vela interior y el hambre de saber qué hay al otro lado del horizonte. Mientras avanza, el origen se desvanece y la promesa de lo desconocido toma forma: la aventura ha comenzado.

Escena Back - Return
Horas o años —la memoria se confunde— después, el regreso es una rueda que vuelve a girar en la misma casa. Hero regresa a Home, no porque haya encontrado todas las respuestas, sino porque el viaje ha sembrado preguntas que sólo la tierra de su hogar puede hacer germinar. Las paredes conocen su nombre y el silencio entre las paredes parece respirar con él, como si el mundo entero hubiera aprendido a 

In [30]:
print(aoi2.content)

A continuación:

- Axis of Interest (en el orden de aparición de sus plot spans)
  1. JOURNEY
  2. RIVALRY
  3. CALL TO ACTION REWARD
  4. REPENTANCE
  5. PURSUIT

- Orden en que se entrelazan los distintos Plot Spans (según la narración):
  1) JOURNEY.Out (Departure)
  2) JOURNEY.Back (Return)
  3) RIVALRY.RivalryDeclared
  4) RIVALRY.Cooperation
  5) RIVALRY.Reconciliation
  6) CALL TO ACTION REWARD.Call
  7) CALL TO ACTION REWARD.Reward
  8) REPENTANCE.Transformation
  9) REPENTANCE.Repentance
  10) REPENTANCE.RepentanceRewarded
  11) PURSUIT.Chase
  12) PURSUIT.Escape

Si quieres, puedo darte una representación visual simple (líneas de tiempo) o un esquema JSON con las relaciones entre plot spans.
